In [5]:
from qiskit_algorithms.utils import algorithm_globals

algorithm_globals.random_seed = 12345 # global seed for reproducibility

In [6]:
from qiskit_machine_learning.datasets import ad_hoc_data

adhoc_dimension = 2
train_features, train_labels, test_features, test_labels, adhoc_total = ad_hoc_data(
    training_size=20,
    test_size=5,
    n=adhoc_dimension,
    gap=0.3,
    plot_data=False,
    one_hot=False,
    include_sample_total=True,
)

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Separate the dataset into features (X) and target label (y)
y = train_labels
X = train_features

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Set parameters for the quantum feature map
feature_dimension = 2  # Number of features used in the quantum feature map
reps = 2  # Number of repetitions of the feature map circuit
entanglement = 'linear'  # Type of entanglement in the quantum circuit

# Define a custom rotational layer for the quantum feature map
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
training_params = ParameterVector("θ", 1)
fm0 = QuantumCircuit(feature_dimension)
for qubit in range(feature_dimension):
    fm0.ry(training_params[0], qubit)

# Use ZZFeatureMap to represent input data
from qiskit.circuit.library import ZZFeatureMap
fm1 = ZZFeatureMap(feature_dimension=feature_dimension, reps=reps, entanglement=entanglement)

# Compose the custom rotational layer with the ZZFeatureMap to create the feature map
fm = fm0.compose(fm1)

# Initialize the Sampler, a Qiskit primitive for sampling from quantum circuits
from qiskit.primitives import Sampler
sampler = Sampler()

# Set up the ComputeUncompute fidelity object for quantum kernel estimation
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel
fidelity = ComputeUncompute(sampler=sampler)

# Instantiate the quantum kernel with the feature map and training parameters
quant_kernel = TrainableFidelityQuantumKernel(fidelity=fidelity, feature_map=fm, training_parameters=training_params)

# Callback class for tracking optimization progress
class QKTCallback:
    # Callback wrapper class
    def __init__(self):
        self._data = [[] for i in range(5)]

    def callback(self, x0, x1=None, x2=None, x3=None, x4=None):
        #Capture callback data for analysis
        for i, x in enumerate([x0, x1, x2, x3, x4]):
            self._data[i].append(x)

    def get_callback_data(self):
        #Get captured callback data
        return self._data

    def clear_callback_data(self):
        #Clear captured callback data
        self._data = [[] for i in range(5)]

# Setup and instantiate the optimizer for the quantum kernel
from qiskit_algorithms.optimizers import SPSA
cb_qkt = QKTCallback()
spsa_opt = SPSA(maxiter=10, callback=cb_qkt.callback, learning_rate=0.01, perturbation=0.05)

# Quantum Kernel Trainer (QKT) for optimizing the kernel parameters
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
qkt = QuantumKernelTrainer(
    quantum_kernel=quant_kernel, loss="svc_loss", optimizer=spsa_opt, initial_point=[np.pi / 2]
)

from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_ = pca.fit_transform(X)

print("Training")

# Train the quantum kernel with the reduced dataset
qka_results = qkt.fit(X_, y)
optimized_kernel = qka_results.quantum_kernel

print("Training complete")

# Use the quantum-enhanced kernel in a Quantum Support Vector Classifier (QSVC)
from qiskit_machine_learning.algorithms import QSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
qsvc = QSVC(quantum_kernel=optimized_kernel)
pipeline = make_pipeline(StandardScaler(), PCA(n_components=2), qsvc)

# Evaluate the performance of the model using cross-validation
from sklearn.model_selection import cross_val_score
cv = cross_val_score(pipeline, X, y, cv=5, n_jobs=1)
mean_score = np.mean(cv)

print(mean_score)

Training
Training complete
0.725


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel
from qiskit_algorithms.optimizers import SPSA
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit_machine_learning.algorithms import QSVC

# Label encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train_labels)

# Quantum feature map setup
feature_dimension = 2
reps = 2
entanglement = 'linear'
training_params = ParameterVector("θ", 1)
fm0 = QuantumCircuit(feature_dimension)
for qubit in range(feature_dimension):
    fm0.ry(training_params[0], qubit)

fm1 = ZZFeatureMap(feature_dimension=feature_dimension, reps=reps, entanglement=entanglement)
fm = fm0.compose(fm1)

# Sampler and fidelity setup for quantum kernel
sampler = Sampler()
fidelity = ComputeUncompute(sampler=sampler)
quant_kernel = TrainableFidelityQuantumKernel(fidelity=fidelity, feature_map=fm, training_parameters=training_params)

# Callback and optimizer for quantum kernel training
class QKTCallback:
    def __init__(self):
        self._data = [[] for i in range(5)]
    def callback(self, x0, x1=None, x2=None, x3=None, x4=None):
        for i, x in enumerate([x0, x1, x2, x3, x4]):
            self._data[i].append(x)
    def get_callback_data(self):
        return self._data
    def clear_callback_data(self):
        self._data = [[] for i in range(5)]

cb_qkt = QKTCallback()
spsa_opt = SPSA(maxiter=10, callback=cb_qkt.callback, learning_rate=0.01, perturbation=0.05)
qkt = QuantumKernelTrainer(quantum_kernel=quant_kernel, loss="svc_loss", optimizer=spsa_opt, initial_point=[np.pi / 2])

# PCA transformation
pca = PCA(n_components=2)
X_ = pca.fit_transform(train_features)

print("Training QSVC")
qka_results = qkt.fit(X_, y)
optimized_kernel = qka_results.quantum_kernel
print("QSVC Training complete")

# Setup QSVC
qsvc = QSVC(quantum_kernel=optimized_kernel)
qsvc_pipeline = make_pipeline(StandardScaler(), PCA(n_components=2), qsvc)

# Evaluate QSVC
qscores = cross_val_score(qsvc_pipeline, train_features, y, cv=5, n_jobs=1)
qmean_score = np.mean(qscores)
print("QSVC Score:", qmean_score)

Training QSVC
QSVC Training complete
QSVC Score: 0.725
SVC Score: 0.375


In [12]:
qsvc.fit(train_features, train_labels)

qsvc_score = qsvc.score(test_features, test_labels)

print(f"QSVC classification test score: {qsvc_score}")

QSVC classification test score: 1.0


In [10]:
# Setup classical SVC
svc = SVC(kernel='rbf')
svc_pipeline = make_pipeline(StandardScaler(), PCA(n_components=2), svc)

# Evaluate classical SVC
cscores = cross_val_score(svc_pipeline, train_features, y, cv=5, n_jobs=1)
cmean_score = np.mean(cscores)
print("SVC Score:", cmean_score)

SVC Score: 0.475
